In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import datetime
from sqlalchemy import create_engine
from pyathena import connect

# show complete dataframe content
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('display.max_colwidth', -1)
# show plots in the notebook
%matplotlib inline

In [2]:
## set up Athena connection
import configparser


config = configparser.ConfigParser()
config.read('/Users/xyin/.aws/credentials')

aws = {key: value for key, value in config.items('default')}

conn = connect(aws_access_key_id=aws['aws_access_key_id'],
                 aws_secret_access_key=aws['aws_secret_access_key'],
                 aws_session_token=aws['aws_session_token'],
                 s3_staging_dir='s3://awsdatasrv-athena-staging/',
                 region_name='us-east-1')

In [3]:
import json

# step 1: import the json of clustering outcome

In [4]:
with open('../../mmr_lookup_profile/metadata/google_output/location_clustering/clusterout_user10_google.json','r') as file:
    google = json.loads(file.read())

In [5]:
import re

# step 2: import cluster result

In [6]:
clusters = pd.read_csv('../metadata/clusterout_user10.csv',sep=',',header=0)

In [7]:
clusters.head(3)

,lat3,long3,cluster_id
0,38.347,-77.488,6
1,33.620,-84.499,1
2,40.886,-81.566,6


In [8]:
auctionlike = clusters.loc[clusters.cluster_id!=4].copy()

In [9]:
auctionlike.dtypes

lat3          float64
long3         float64
cluster_id    int64  
dtype: object

# step 3: create georgia geofence

In [10]:
georgia = [
    (34.992756,-85.625226),
    (30.721736,-84.926279),
    (30.589405,-81.492173),
    (32.016373,-80.817001),
    (34.992871,-83.101132),
    (34.992756,-85.625226)
]

In [11]:
import folium
from folium import plugins


import gmplot
from gmplot import gmplot2

In [12]:
m = folium.Map([33.909508, -84.355094], zoom_start=5)
folium.PolyLine(georgia).add_to(m)
m

# step 4: find out the georgia clusters

In [13]:
auctionlike_latlong = [(lat,long) for lat,long in zip(auctionlike.lat3,auctionlike.long3)]

In [14]:
auctionlike_latlong[:5]

[(38.347, -77.488),
 (33.62, -84.499),
 (40.885999999999996, -81.566),
 (33.865, -117.84100000000001),
 (34.202, -118.402)]

In [15]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [16]:
polygon = Polygon(georgia)

In [17]:
auctionlike_georgia = [record for record in auctionlike_latlong if polygon.contains(Point(record[0],record[1]))]

In [18]:
len(auctionlike_georgia)

212

In [19]:
m = folium.Map([33.909508, -84.355094], zoom_start=6)

# mark all cluster locations
for location in auctionlike_georgia:
    folium.Marker(
        [location[0], location[1]],
#         popup=folium.Popup(row['cluster_id'],parse_html=True),
        icon=folium.Icon(color='red')
    ).add_to(m)
    
m.save('../metadata/test.html')
m

# step 5: find the corresponding nearby business found in google

In [20]:
auctionlike_georgia_dict_raw = {
    str(round(record[0],3))+','+str(round(record[1],3)):google[str(round(record[0],3))+','+str(round(record[1],3))][0]
    for record in auctionlike_georgia if (len(google[str(round(record[0],3))+','+str(round(record[1],3))])>0)
}

In [21]:
auctionlike_georgia_dict = {
    key:{
        'name':value['name'],
        'latlong':"{},{}".format(value['geometry']['location']['lat'],value['geometry']['location']['lng']),
        'address':value['formatted_address']
    }
    for key,value in auctionlike_georgia_dict_raw.items() if bool(re.match('.*auto.*auction.*',value['name'].lower()))
}

In [22]:
auctionlike_georgia_dict

{'32.652,-83.648': {'name': 'VEMO Auto Auctions',
  'latlong': '32.6510009,-83.64865979999999',
  'address': '441 Dunbar Rd, Warner Robins, GA 31093, USA'},
 '34.234,-83.898': {'name': 'Oakwoods Arrow Auto Auction',
  'latlong': '34.2334842,-83.8984545',
  'address': '4712 Flat Creek Rd, Oakwood, GA 30566, USA'},
 '33.629,-84.305': {'name': 'AutoNation Auto Auction Atlanta',
  'latlong': '33.6303677,-84.3080105',
  'address': '2491 Old Anvil Block Rd, Ellenwood, GA 30294, USA'},
 '34.259,-83.489': {'name': 'Georgia - Carolina Auto Auction',
  'latlong': '34.2589807,-83.4892078',
  'address': '884 E Ridgeway Rd, Commerce, GA 30529, USA'},
 '32.651,-83.649': {'name': 'VEMO Auto Auctions',
  'latlong': '32.6510009,-83.64865979999999',
  'address': '441 Dunbar Rd, Warner Robins, GA 31093, USA'},
 '34.233,-83.899': {'name': 'Oakwoods Arrow Auto Auction',
  'latlong': '34.2334842,-83.8984545',
  'address': '4712 Flat Creek Rd, Oakwood, GA 30566, USA'},
 '33.933,-84.17': {'name': 'Auto Auctio

In [23]:
auctions_new = set([
    value['name']+'|'+value['latlong']+'|'+value['address'].replace(', USA','')
    for value in auctionlike_georgia_dict.values()
])

In [24]:
auctions_new

{'Albany Auto Auction|31.5092048,-84.1166549|1421 Liberty Expy, Albany, GA 31705',
 'Athens Auto Auction|33.9443602,-83.507815|5050 Atlanta Hwy, Bogart, GA 30622',
 'Augusta Auto Auction|33.4827023,-81.9460647|395 Hamburg Rd, North Augusta, SC 29841',
 'Auto Auction Services Corporation|33.9405501,-84.361437|50 Glenlake Pkwy NE, Atlanta, GA 30328',
 'Auto Auctions|33.954841,-84.198661|2775 Northwoods Pkwy, Norcross, GA 30071',
 'AutoNation Auto Auction Atlanta|33.6303677,-84.3080105|2491 Old Anvil Block Rd, Ellenwood, GA 30294',
 'Georgia - Carolina Auto Auction|34.2589807,-83.4892078|884 E Ridgeway Rd, Commerce, GA 30529',
 'Hwy 515 Auto Auction|34.741224,-84.42113300000001|107 Whitepath Rd, Ellijay, GA 30540',
 'Oakwoods Arrow Auto Auction|34.2334842,-83.8984545|4712 Flat Creek Rd, Oakwood, GA 30566',
 'Southeastern Auto Auction|32.0867058,-81.1966333|1712 Dean Forest Rd, Savannah, GA 31408',
 'VEMO Auto Auctions|32.6510009,-83.64865979999999|441 Dunbar Rd, Warner Robins, GA 31093'}

In [25]:
m = folium.Map([33.909508, -84.355094], zoom_start=6)

# mark all cluster locations
for location in auctionlike_georgia:
    folium.Marker(
        [location[0], location[1]],
        icon=folium.Icon(color='red')
    ).add_to(m)

# mark all the auctions found via google
for location in auctions_new:
      folium.Marker(
        [float(location.split('|')[1].split(',')[0]), float(location.split('|')[1].split(',')[1])],
        popup=folium.Popup(location.split('|')[0],parse_html=True),
        icon=folium.Icon(color='green')
    ).add_to(m)  

m.save('../metadata/test.html')
m

# step 6: import known list of auction locations

In [26]:
query = '''
SELECT
  location_cat,
  location_name,
  addr,
  lat,
  long
FROM default.ds_geo_poi
WHERE location_cat IN ('manheim','competitor auctions');
'''

auctions = pd.read_sql(query, conn)
auctions.lat = auctions.lat.astype(float)
auctions.long = auctions.long.astype(float)

In [27]:
auctions.head(3)

,location_cat,location_name,addr,lat,long
0,competitor auctions,166 AUTO AUCTION,"2944 WEST SUNSHINE ST, SPRINGFIELD, MO 65807",37.181888,-93.337763
1,competitor auctions,411 AUTO AUCTION,"3824 HIGHWAY 411, KINGSTON, GA 30145",34.201204,-84.998695
2,competitor auctions,AAA ALLIANCE AUTO AUCTION- DALLAS,"9426 LAKEFIELD BLVD, DALLAS, TX 75220",32.858631,-96.862701


In [28]:
auctions_georgia = auctions.loc[auctions.addr.map(lambda x: ' GA ' in x)]
auctions_georgia = auctions_georgia.reset_index().drop(columns='index')
auctions_georgia

,location_cat,location_name,addr,lat,long
0,competitor auctions,411 AUTO AUCTION,"3824 HIGHWAY 411, KINGSTON, GA 30145",34.201204,-84.998695
1,competitor auctions,ADESA ATLANTA,"5055 OAKLEY INDUSTRIAL BLVD, FAIRBURN, GA 30213",33.547412,-84.549661
2,competitor auctions,AMERICA'S AUTO AUCTION ATLANTA,"444 JOE FRANK HARRIS PKWY, CARTERSVILLE, GA 30120",34.145539,-84.745675
3,competitor auctions,CARMAX NORCROSS,"1975 BEAVER RUIN RD, NORCROSS, GA 30071",33.932711,-84.171008
4,competitor auctions,CARMAX STOCKBRIDGE,"3100 MT. ZION PKWY, STOCKBRIDGE, GA 30281",33.549901,-84.289497
5,competitor auctions,CARMAX KENNESAW,"1215 ERNEST BARRETT PKWY, KENNESAW, GA 30144",33.998176,-84.584873
6,competitor auctions,COPART ATLANTA EAST/WEST/SOUTH/NORTH,"6089 HIGHWAY 20, LOGANVILLE, GA 30052",33.804355,-83.955132
7,competitor auctions,COPART CARTERSVILLE,"1880 HWY 113, CARTERSVILLE, GA 30120",34.117550,-84.892990
8,competitor auctions,COPART SAVANNAH,"5510 SILK HOPE ROAD, SAVANNAH, GA 31405",32.040761,-81.212716
9,competitor auctions,COPART TIFTON,"399 OAKRIDGE CHURCH RD, TIFTON, GA 31794",31.406281,-83.487636


In [29]:
auctions_georgia_flat = [str(name)+'|'+str(lat)+','+str(long)
                         for name,lat,long in zip(auctions_georgia.location_name,auctions_georgia.lat,auctions_georgia.long)
                        ]

In [30]:
auctions_georgia_flat

['411 AUTO AUCTION|34.2012036,-84.998695',
 'ADESA ATLANTA|33.5474124,-84.5496607',
 "AMERICA'S AUTO AUCTION ATLANTA|34.1455393,-84.7456754",
 'CARMAX NORCROSS|33.9327112,-84.1710076',
 'CARMAX STOCKBRIDGE|33.5499008,-84.2894974',
 'CARMAX KENNESAW|33.9981764,-84.5848731',
 'COPART ATLANTA EAST/WEST/SOUTH/NORTH|33.8043554,-83.9551319',
 'COPART CARTERSVILLE|34.1175502,-84.8929903',
 'COPART SAVANNAH|32.0407614,-81.2127155',
 'COPART TIFTON|31.406281,-83.487636',
 'IAA ATLANTA/EAST/NORTH/SOUTH|33.7249363,-83.912646',
 'IAA MACON|32.848037,-83.584819',
 'IAA SAVANNAH|32.2460172,-81.2013731',
 'IAA TIFTON|31.4005578,-83.4899801',
 'MANHEIM ATLANTA|33.6192003,-84.49488',
 'MANHEIM GEORGIA|33.6947088,-84.6228706',
 'MANHEIM METRO ATLANTA|33.6921367,-84.4059195',
 'SOUTHEASTERN AUTO AUCTION OF SAVANNAH|32.0867103,-81.1966333',
 'V.I.P AUTO AUCTION ATLANTA|34.09157,-84.0067847',
 'VEMO WARNER ROBBINS|32.652433,-83.648684',
 'WESTWOOD AUTO AUCTION|33.6083878,-85.017966']

In [31]:
len(auctions_georgia_flat)

21

# step 7: identify the new locations

In [32]:
auctions_new = sorted(list(auctions_new))
auctions_new[:9]

['Albany Auto Auction|31.5092048,-84.1166549|1421 Liberty Expy, Albany, GA 31705',
 'Athens Auto Auction|33.9443602,-83.507815|5050 Atlanta Hwy, Bogart, GA 30622',
 'Augusta Auto Auction|33.4827023,-81.9460647|395 Hamburg Rd, North Augusta, SC 29841',
 'Auto Auction Services Corporation|33.9405501,-84.361437|50 Glenlake Pkwy NE, Atlanta, GA 30328',
 'Auto Auctions|33.954841,-84.198661|2775 Northwoods Pkwy, Norcross, GA 30071',
 'AutoNation Auto Auction Atlanta|33.6303677,-84.3080105|2491 Old Anvil Block Rd, Ellenwood, GA 30294',
 'Georgia - Carolina Auto Auction|34.2589807,-83.4892078|884 E Ridgeway Rd, Commerce, GA 30529',
 'Hwy 515 Auto Auction|34.741224,-84.42113300000001|107 Whitepath Rd, Ellijay, GA 30540',
 'Oakwoods Arrow Auto Auction|34.2334842,-83.8984545|4712 Flat Creek Rd, Oakwood, GA 30566']

In [33]:
auctions_new.pop(3)
auctions_new.pop(3)

'Auto Auctions|33.954841,-84.198661|2775 Northwoods Pkwy, Norcross, GA 30071'

In [34]:
auctions_new

['Albany Auto Auction|31.5092048,-84.1166549|1421 Liberty Expy, Albany, GA 31705',
 'Athens Auto Auction|33.9443602,-83.507815|5050 Atlanta Hwy, Bogart, GA 30622',
 'Augusta Auto Auction|33.4827023,-81.9460647|395 Hamburg Rd, North Augusta, SC 29841',
 'AutoNation Auto Auction Atlanta|33.6303677,-84.3080105|2491 Old Anvil Block Rd, Ellenwood, GA 30294',
 'Georgia - Carolina Auto Auction|34.2589807,-83.4892078|884 E Ridgeway Rd, Commerce, GA 30529',
 'Hwy 515 Auto Auction|34.741224,-84.42113300000001|107 Whitepath Rd, Ellijay, GA 30540',
 'Oakwoods Arrow Auto Auction|34.2334842,-83.8984545|4712 Flat Creek Rd, Oakwood, GA 30566',
 'Southeastern Auto Auction|32.0867058,-81.1966333|1712 Dean Forest Rd, Savannah, GA 31408',
 'VEMO Auto Auctions|32.6510009,-83.64865979999999|441 Dunbar Rd, Warner Robins, GA 31093']

In [35]:
m = folium.Map([33.909508, -84.355094], zoom_start=6)

# mark all cluster locations
for location in auctionlike_georgia:
    folium.Marker(
        [location[0], location[1]],
        icon=folium.Icon(color='red')
    ).add_to(m)

# mark all the auctions found via google
for location in auctions_new[:9]:
      folium.Marker(
        [float(location.split('|')[1].split(',')[0]), float(location.split('|')[1].split(',')[1])],
        popup=folium.Popup(location.split('|')[0],parse_html=True),
        icon=folium.Icon(color='green')
    ).add_to(m)  

# mark all the known auctions in Georgia
for location in auctions_georgia_flat:
      folium.Marker(
        [float(location.split('|')[1].split(',')[0]), float(location.split('|')[1].split(',')[1])],
        popup=folium.Popup(location.split('|')[0],parse_html=True),
        icon=folium.Icon(color='blue')
    ).add_to(m)  

m.save('../metadata/test.html')
m

# step 8: elbow plot for Georgia blobs

In [36]:
auctions_new

['Albany Auto Auction|31.5092048,-84.1166549|1421 Liberty Expy, Albany, GA 31705',
 'Athens Auto Auction|33.9443602,-83.507815|5050 Atlanta Hwy, Bogart, GA 30622',
 'Augusta Auto Auction|33.4827023,-81.9460647|395 Hamburg Rd, North Augusta, SC 29841',
 'AutoNation Auto Auction Atlanta|33.6303677,-84.3080105|2491 Old Anvil Block Rd, Ellenwood, GA 30294',
 'Georgia - Carolina Auto Auction|34.2589807,-83.4892078|884 E Ridgeway Rd, Commerce, GA 30529',
 'Hwy 515 Auto Auction|34.741224,-84.42113300000001|107 Whitepath Rd, Ellijay, GA 30540',
 'Oakwoods Arrow Auto Auction|34.2334842,-83.8984545|4712 Flat Creek Rd, Oakwood, GA 30566',
 'Southeastern Auto Auction|32.0867058,-81.1966333|1712 Dean Forest Rd, Savannah, GA 31408',
 'VEMO Auto Auctions|32.6510009,-83.64865979999999|441 Dunbar Rd, Warner Robins, GA 31093']

In [37]:
auctionlike.head(3)

,lat3,long3,cluster_id
0,38.347,-77.488,6
1,33.620,-84.499,1
2,40.886,-81.566,6


In [38]:
auctionlike_df = pd.DataFrame(auctionlike_georgia, columns=['lat','long'])
auctionlike_df.shape

(212, 2)

## 8.2: append auction list

In [39]:
auctions_new = auctions_new[:7]

In [40]:
auctions_new

['Albany Auto Auction|31.5092048,-84.1166549|1421 Liberty Expy, Albany, GA 31705',
 'Athens Auto Auction|33.9443602,-83.507815|5050 Atlanta Hwy, Bogart, GA 30622',
 'Augusta Auto Auction|33.4827023,-81.9460647|395 Hamburg Rd, North Augusta, SC 29841',
 'AutoNation Auto Auction Atlanta|33.6303677,-84.3080105|2491 Old Anvil Block Rd, Ellenwood, GA 30294',
 'Georgia - Carolina Auto Auction|34.2589807,-83.4892078|884 E Ridgeway Rd, Commerce, GA 30529',
 'Hwy 515 Auto Auction|34.741224,-84.42113300000001|107 Whitepath Rd, Ellijay, GA 30540',
 'Oakwoods Arrow Auto Auction|34.2334842,-83.8984545|4712 Flat Creek Rd, Oakwood, GA 30566']

In [41]:
auctions_georgia_flat

['411 AUTO AUCTION|34.2012036,-84.998695',
 'ADESA ATLANTA|33.5474124,-84.5496607',
 "AMERICA'S AUTO AUCTION ATLANTA|34.1455393,-84.7456754",
 'CARMAX NORCROSS|33.9327112,-84.1710076',
 'CARMAX STOCKBRIDGE|33.5499008,-84.2894974',
 'CARMAX KENNESAW|33.9981764,-84.5848731',
 'COPART ATLANTA EAST/WEST/SOUTH/NORTH|33.8043554,-83.9551319',
 'COPART CARTERSVILLE|34.1175502,-84.8929903',
 'COPART SAVANNAH|32.0407614,-81.2127155',
 'COPART TIFTON|31.406281,-83.487636',
 'IAA ATLANTA/EAST/NORTH/SOUTH|33.7249363,-83.912646',
 'IAA MACON|32.848037,-83.584819',
 'IAA SAVANNAH|32.2460172,-81.2013731',
 'IAA TIFTON|31.4005578,-83.4899801',
 'MANHEIM ATLANTA|33.6192003,-84.49488',
 'MANHEIM GEORGIA|33.6947088,-84.6228706',
 'MANHEIM METRO ATLANTA|33.6921367,-84.4059195',
 'SOUTHEASTERN AUTO AUCTION OF SAVANNAH|32.0867103,-81.1966333',
 'V.I.P AUTO AUCTION ATLANTA|34.09157,-84.0067847',
 'VEMO WARNER ROBBINS|32.652433,-83.648684',
 'WESTWOOD AUTO AUCTION|33.6083878,-85.017966']

In [42]:
len(auctions_georgia_flat)

21

In [43]:
auctions_known = pd.DataFrame([
    (
        location.split('|')[1].split(',')[0],
        location.split('|')[1].split(',')[1],
        location.split('|')[0]
    )
    for location in auctions_georgia_flat
], columns=['lat','long','name'])

auctions_new2 = pd.DataFrame([
    (
        location.split('|')[1].split(',')[0],
        location.split('|')[1].split(',')[1],
        location.split('|')[0]
    )
    for location in auctions_new
], columns=['lat','long','name'])

In [44]:
auctionlike_df['name']='Unknown'

In [45]:
auctionlike_df2 = pd.concat([
    auctionlike_df,
    auctions_known,
    auctions_new2
], ignore_index=True)

auctionlike_df2.lat = auctionlike_df2.lat.map(lambda x: float(x))
auctionlike_df2.long = auctionlike_df2.long.map(lambda x: float(x))

In [46]:
auctionlike_df2.head(3)

,lat,long,name
0,33.620,-84.499,Unknown
1,34.147,-84.746,Unknown
2,32.652,-83.648,Unknown


In [47]:
auctionlist = auctionlike_df2.loc[auctionlike_df2.name!='Unknown'].copy()

In [48]:
auctionlist

,lat,long,name
212,34.201204,-84.998695,411 AUTO AUCTION
213,33.547412,-84.549661,ADESA ATLANTA
214,34.145539,-84.745675,AMERICA'S AUTO AUCTION ATLANTA
215,33.932711,-84.171008,CARMAX NORCROSS
216,33.549901,-84.289497,CARMAX STOCKBRIDGE
217,33.998176,-84.584873,CARMAX KENNESAW
218,33.804355,-83.955132,COPART ATLANTA EAST/WEST/SOUTH/NORTH
219,34.117550,-84.892990,COPART CARTERSVILLE
220,32.040761,-81.212716,COPART SAVANNAH
221,31.406281,-83.487636,COPART TIFTON


In [49]:
auctionlist['lat_low'] = auctionlist.lat.map(lambda x: round(x,2)-0.02)
auctionlist['lat_high'] = auctionlist.lat.map(lambda x: round(x,2)+0.02)
auctionlist['long_low'] = auctionlist.long.map(lambda x: round(x,2)-0.02)
auctionlist['long_high'] = auctionlist.long.map(lambda x: round(x,2)+0.02)

In [50]:
auctionlist.head(3)

,lat,long,name,lat_low,lat_high,long_low,long_high
212,34.201204,-84.998695,411 AUTO AUCTION,34.18,34.22,-85.02,-84.98
213,33.547412,-84.549661,ADESA ATLANTA,33.53,33.57,-84.57,-84.53
214,34.145539,-84.745675,AMERICA'S AUTO AUCTION ATLANTA,34.13,34.17,-84.77,-84.73


In [51]:
auctionlist.name.nunique()

28

In [52]:
auctionlist['comm'] = 1
unknowns = auctionlike_df2.copy()
unknowns['comm'] = 1


combo = pd.merge(
    auctionlist.rename(columns={'lat':'lat_poi','long':'long_poi'}),
    unknowns.rename(columns={'name':'label'}),
    on='comm'
)

combo.head(3)

,lat_poi,long_poi,name,lat_low,lat_high,long_low,long_high,comm,lat,long,label
0,34.201204,-84.998695,411 AUTO AUCTION,34.18,34.22,-85.02,-84.98,1,33.620,-84.499,Unknown
1,34.201204,-84.998695,411 AUTO AUCTION,34.18,34.22,-85.02,-84.98,1,34.147,-84.746,Unknown
2,34.201204,-84.998695,411 AUTO AUCTION,34.18,34.22,-85.02,-84.98,1,32.652,-83.648,Unknown


In [53]:
combo2 = combo.loc[
    (combo.lat>=combo.lat_low)\
    &(combo.lat<=combo.lat_high)\
    &(combo.long>=combo.long_low)\
    &(combo.long<=combo.long_high)
]

In [54]:
combo2.shape

(220, 11)

In [55]:
combo2_agg_p1 = combo2.groupby(['name','lat_poi','long_poi'])[['lat','long']].min().reset_index().rename(
    columns={
        'lat':'southwest_lat',
        'long':'southwest_long'
    }
)
combo2_agg_p2 = combo2.groupby(['name','lat_poi','long_poi'])[['lat','long']].max().reset_index().rename(
    columns={
        'lat':'northeast_lat',
        'long':'northeast_long'
    }
)

combo2_agg = pd.merge(
    combo2_agg_p1,
    combo2_agg_p2,
    on=['name','lat_poi','long_poi']
)

In [56]:
combo2_agg

,name,lat_poi,long_poi,southwest_lat,southwest_long,northeast_lat,northeast_long
0,411 AUTO AUCTION,34.201204,-84.998695,34.201204,-84.998695,34.201204,-84.998695
1,ADESA ATLANTA,33.547412,-84.549661,33.537000,-84.567000,33.548000,-84.548000
2,AMERICA'S AUTO AUCTION ATLANTA,34.145539,-84.745675,34.140000,-84.758000,34.168000,-84.744000
3,Albany Auto Auction,31.509205,-84.116655,31.509000,-84.116655,31.509205,-84.116000
4,Athens Auto Auction,33.944360,-83.507815,33.944000,-83.509000,33.944360,-83.506000
5,Augusta Auto Auction,33.482702,-81.946065,33.482000,-81.947000,33.484000,-81.946000
6,AutoNation Auto Auction Atlanta,33.630368,-84.308010,33.629000,-84.308010,33.631000,-84.305000
7,CARMAX KENNESAW,33.998176,-84.584873,33.998000,-84.585000,33.998176,-84.583000
8,CARMAX NORCROSS,33.932711,-84.171008,33.932000,-84.171008,33.933000,-84.167000
9,CARMAX STOCKBRIDGE,33.549901,-84.289497,33.549000,-84.291000,33.550000,-84.289497


In [57]:
combo2_agg_copy = combo2_agg.copy()

combo2_agg.loc[
    (combo2_agg_copy.southwest_lat>combo2_agg_copy.northeast_lat-0.001),
    'southwest_lat'
] = combo2_agg.loc[
    (combo2_agg_copy.southwest_lat>combo2_agg_copy.northeast_lat-0.001),
    'lat_poi'
] - 0.001

combo2_agg.loc[
    (combo2_agg_copy.southwest_lat>combo2_agg_copy.northeast_lat-0.001),
    'northeast_lat'
] = combo2_agg.loc[
    (combo2_agg_copy.southwest_lat>combo2_agg_copy.northeast_lat-0.001),
    'lat_poi'
] + 0.001

combo2_agg.loc[
    (combo2_agg_copy.southwest_long>combo2_agg_copy.northeast_long-0.001),
    'southwest_long'
] = combo2_agg.loc[
    (combo2_agg_copy.southwest_long>combo2_agg_copy.northeast_long-0.001),
    'long_poi'
] - 0.001

combo2_agg.loc[
    (combo2_agg_copy.southwest_long>combo2_agg_copy.northeast_long-0.001),
    'northeast_long'
] = combo2_agg.loc[
    (combo2_agg_copy.southwest_long>combo2_agg_copy.northeast_long-0.001),
    'long_poi'
] + 0.001

In [59]:
combo2_agg

,name,lat_poi,long_poi,southwest_lat,southwest_long,northeast_lat,northeast_long
0,411 AUTO AUCTION,34.201204,-84.998695,34.200204,-84.999695,34.202204,-84.997695
1,ADESA ATLANTA,33.547412,-84.549661,33.537000,-84.567000,33.548000,-84.548000
2,AMERICA'S AUTO AUCTION ATLANTA,34.145539,-84.745675,34.140000,-84.758000,34.168000,-84.744000
3,Albany Auto Auction,31.509205,-84.116655,31.508205,-84.117655,31.510205,-84.115655
4,Athens Auto Auction,33.944360,-83.507815,33.943360,-83.509000,33.945360,-83.506000
5,Augusta Auto Auction,33.482702,-81.946065,33.482000,-81.947000,33.484000,-81.946000
6,AutoNation Auto Auction Atlanta,33.630368,-84.308010,33.629000,-84.308010,33.631000,-84.305000
7,CARMAX KENNESAW,33.998176,-84.584873,33.997176,-84.585000,33.999176,-84.583000
8,CARMAX NORCROSS,33.932711,-84.171008,33.932000,-84.171008,33.933000,-84.167000
9,CARMAX STOCKBRIDGE,33.549901,-84.289497,33.549000,-84.291000,33.550000,-84.289497


In [60]:
combo2_agg['fence'] = [
    [
        (southwest_lat,southwest_long),
        (southwest_lat,northeast_long),
        (northeast_lat,northeast_long),
        (northeast_lat,southwest_long),
        (southwest_lat,southwest_long)
    ]
    for southwest_lat,southwest_long,northeast_lat,northeast_long in zip(
        combo2_agg.southwest_lat,
        combo2_agg.southwest_long,
        combo2_agg.northeast_lat,
        combo2_agg.northeast_long
    )
]

In [61]:
combo2_agg

,name,lat_poi,long_poi,southwest_lat,southwest_long,northeast_lat,northeast_long,fence
0,411 AUTO AUCTION,34.201204,-84.998695,34.200204,-84.999695,34.202204,-84.997695,"[(34.2002036, -84.999695), (34.2002036, -84.997695), (34.2022036, -84.997695), (34.2022036, -84.999695), (34.2002036, -84.999695)]"
1,ADESA ATLANTA,33.547412,-84.549661,33.537000,-84.567000,33.548000,-84.548000,"[(33.537, -84.56700000000001), (33.537, -84.54799999999999), (33.548, -84.54799999999999), (33.548, -84.56700000000001), (33.537, -84.56700000000001)]"
2,AMERICA'S AUTO AUCTION ATLANTA,34.145539,-84.745675,34.140000,-84.758000,34.168000,-84.744000,"[(34.14, -84.758), (34.14, -84.744), (34.168, -84.744), (34.168, -84.758), (34.14, -84.758)]"
3,Albany Auto Auction,31.509205,-84.116655,31.508205,-84.117655,31.510205,-84.115655,"[(31.508204799999998, -84.1176549), (31.508204799999998, -84.1156549), (31.5102048, -84.1156549), (31.5102048, -84.1176549), (31.508204799999998, -84.1176549)]"
4,Athens Auto Auction,33.944360,-83.507815,33.943360,-83.509000,33.945360,-83.506000,"[(33.9433602, -83.509), (33.9433602, -83.506), (33.945360199999996, -83.506), (33.945360199999996, -83.509), (33.9433602, -83.509)]"
5,Augusta Auto Auction,33.482702,-81.946065,33.482000,-81.947000,33.484000,-81.946000,"[(33.482, -81.947), (33.482, -81.946), (33.484, -81.946), (33.484, -81.947), (33.482, -81.947)]"
6,AutoNation Auto Auction Atlanta,33.630368,-84.308010,33.629000,-84.308010,33.631000,-84.305000,"[(33.629, -84.3080105), (33.629, -84.305), (33.631, -84.305), (33.631, -84.3080105), (33.629, -84.3080105)]"
7,CARMAX KENNESAW,33.998176,-84.584873,33.997176,-84.585000,33.999176,-84.583000,"[(33.9971764, -84.585), (33.9971764, -84.583), (33.999176399999996, -84.583), (33.999176399999996, -84.585), (33.9971764, -84.585)]"
8,CARMAX NORCROSS,33.932711,-84.171008,33.932000,-84.171008,33.933000,-84.167000,"[(33.931999999999995, -84.1710076), (33.931999999999995, -84.167), (33.933, -84.167), (33.933, -84.1710076), (33.931999999999995, -84.1710076)]"
9,CARMAX STOCKBRIDGE,33.549901,-84.289497,33.549000,-84.291000,33.550000,-84.289497,"[(33.549, -84.291), (33.549, -84.2894974), (33.55, -84.2894974), (33.55, -84.291), (33.549, -84.291)]"


In [62]:
m = folium.Map([33.909508, -84.355094], zoom_start=6)

# mark all cluster locations
for location in auctionlike_georgia:
    folium.Marker(
        [location[0], location[1]],
        icon=folium.Icon(color='red')
    ).add_to(m)

# mark all the auctions found via google
for location in auctions_new[:7]:
      folium.Marker(
        [float(location.split('|')[1].split(',')[0]), float(location.split('|')[1].split(',')[1])],
        popup=folium.Popup(location.split('|')[0],parse_html=True),
        icon=folium.Icon(color='green')
    ).add_to(m)  

# mark all the known auctions in Georgia
for location in auctions_georgia_flat:
      folium.Marker(
        [float(location.split('|')[1].split(',')[0]), float(location.split('|')[1].split(',')[1])],
        popup=folium.Popup(location.split('|')[0],parse_html=True),
        icon=folium.Icon(color='blue')
    ).add_to(m)  
        
# mark the geofences
for fence in combo2_agg.fence:
    folium.PolyLine(fence).add_to(m)

m.save('../metadata/test2.html')
m

In [64]:
## visualize with sattelite image

colors = ['white','megenta','yellow','green','blue','purple']
gmap = gmplot2.GoogleMapPlotter(33.909508, -84.355094, 5)

# mark all cluster locations
for location in auctionlike_georgia:
    gmap.marker(location[0], location[1],'red')

# mark all the auctions found via google
for location in auctions_new[:7]: 
    gmap.marker(
        float(location.split('|')[1].split(',')[0]), 
        float(location.split('|')[1].split(',')[1]),
        'green',
        title=location.split('|')[0]
    )
# mark all the known auctions in Georgia
for location in auctions_georgia_flat:
    gmap.marker(
        float(location.split('|')[1].split(',')[0]), 
        float(location.split('|')[1].split(',')[1]),
        'blue',
        title=location.split('|')[0]
    )

# mark the geofences!!
# Polygon
for fence in combo2_agg.fence:
    lats_fence, lons_fence = zip(*fence)
    gmap.plot(lats_fence, lons_fence, 'm', edge_width=10)

# mark the georgia state geofence
lats_fence, lons_fence = zip(*georgia)
gmap.plot(lats_fence, lons_fence, 'm', edge_width=5)

# Generate the heatmap into an HTML file
gmap.draw("../metadata/test2_wsattelite.html")

## 8.3: save the geofences

In [63]:
combo2_agg.to_csv(
    '../metadata/tb_geofence_georgia.csv',
    sep=',',
    index=False
)

# step 9: import the vinscan data

In [64]:
scans = pd.read_csv(
    '../../mmr_lookup_profile/metadata/latlong3_frequency_wvar_behavior.csv', 
    header=0, 
    sep=','
)
scans.head(3)

,lat3,long3,scan_mo_1,user_mo_1,vin_mo_1,day_mo_1,scan_mo_2,user_mo_2,vin_mo_2,day_mo_2,scan_mo_3,user_mo_3,vin_mo_3,day_mo_3,scan_mo_6,user_mo_6,vin_mo_6,day_mo_6,scan_mo_12,user_mo_12,vin_mo_12,day_mo_12,scan_mo_all,user_mo_all,vin_mo_all,day_mo_all,user_1yr_var,scan_1yr_var,vin_1yr_var
0,28.079,-80.616,15,1,14,8,25,2,23,14,40,1,40,18,148,1,133,54,118,1,116,72,487,4,458,249,0.006024,2.392844,1.908762
1,42.017,-80.243,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,6,3,6,3,0.000000,0.000000,0.000000
2,38.347,-77.488,42,25,42,8,48,24,42,10,67,27,61,10,153,62,147,24,242,85,225,50,1116,270,1024,164,3.326344,19.709959,17.193943


## 9.1: filter out locations within the georgia auction geofences